In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.insert(0, project_root)

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *

s_raw_root_path = r'F:\DataSamples\DataSets'
s_spark_file_server_root = r'F:\Spark_Data_Test'

spark = SparkSession.builder \
    .appName('Prep Census Data') \
    .master("local[*]") \
    .config("spark.sql.warehouse.dir", s_spark_file_server_root) \
    .enableHiveSupport() \
    .getOrCreate()


In [24]:
schema = StructType([
    StructField('FirstName', StringType(), False),
    StructField('Gender', StringType(), True),
    StructField('Count', IntegerType(), True)
])

# df_ssa_names = spark.read.options(delimiter=',', enforceSchema=True).schema(schema).load(os.path.join(s_raw_root_path, 'SSA_Names\\*.txt'), format='csv')
df_ssa_names = spark.read.format('csv').options(delimiter=',', enforceSchema=True).schema(schema).load(os.path.join(s_raw_root_path, 'SSA_Names\\*.txt'))
df_ssa_names.show(10, False)


+---------+------+-----+
|FirstName|Gender|Count|
+---------+------+-----+
|Emma     |F     |18826|
|Isabella |F     |18626|
|Emily    |F     |17444|
|Olivia   |F     |17087|
|Ava      |F     |17049|
|Madison  |F     |17046|
|Sophia   |F     |16090|
|Abigail  |F     |15085|
|Elizabeth|F     |12025|
|Chloe    |F     |11833|
+---------+------+-----+
only showing top 10 rows



In [ ]:
df_ssa_names.printSchema()
# 2,117,219
df_ssa_names.count()

root
 |-- FirstName: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Count: integer (nullable = true)



2117219

In [38]:
df_ssa_names_w_file = df_ssa_names.withColumn('FileName', input_file_name()) \
    .withColumn('YYYY', regexp_extract(col('FileName'), r'(\d{4})', 1).cast(IntegerType())) \

df_ssa_names_w_file.show(10, False)

+---------+------+-----+-----------------------------------------------------+----+
|FirstName|Gender|Count|FileName                                             |YYYY|
+---------+------+-----+-----------------------------------------------------+----+
|Emma     |F     |18826|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Isabella |F     |18626|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Emily    |F     |17444|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Olivia   |F     |17087|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Ava      |F     |17049|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Madison  |F     |17046|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Sophia   |F     |16090|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Abigail  |F     |15085|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.txt|2008|
|Elizabeth|F     |12025|file:///F:/DataSamples/DataSets/SSA_Names/yob2008.tx

In [40]:
df_ssa_names_w_file.select(min('YYYY'), max('YYYY')
                           , countDistinct('YYYY') 
                           , countDistinct('FileName') 
                           , countDistinct('Gender')
                           ).show(1, False)

+---------+---------+--------------------+------------------------+----------------------+
|min(YYYY)|max(YYYY)|count(DISTINCT YYYY)|count(DISTINCT FileName)|count(DISTINCT Gender)|
+---------+---------+--------------------+------------------------+----------------------+
|1880     |2023     |144                 |144                     |2                     |
+---------+---------+--------------------+------------------------+----------------------+



In [43]:
# Filter
df_names_agg = df_ssa_names_w_file.filter(col('YYYY') >= 1950) \
    .groupBy('FirstName') \
        .agg(
            sum('Count').alias('TotalCount'),
            countDistinct('YYYY').alias('YearCount')
    )

df_names_agg.show(10, False)

+---------+----------+---------+
|FirstName|TotalCount|YearCount|
+---------+----------+---------+
|Rosendo  |3989      |74       |
|Shyana   |264       |24       |
|Ritu     |341       |41       |
|Irais    |611       |42       |
|Jaliana  |85        |13       |
|Sira     |232       |25       |
|Zolie    |60        |9        |
|Syris    |328       |25       |
|Chandni  |400       |23       |
|Annajulia|60        |10       |
+---------+----------+---------+
only showing top 10 rows



In [ ]:
i_sum = df_names_agg.agg(sum('TotalCount')).collect()[0][0]
# Total: 270,176,741
print(f'Total: {i_sum:,}')


Total: 270,176,741


In [48]:
from pyspark.sql import Window
df_names_agg_stats = df_names_agg.withColumn('frequency', col('TotalCount')/i_sum) \
    .withColumn('overall_rank', row_number().over(Window.orderBy(col('TotalCount').desc()))) \
    .withColumn('freq_dense_rank', dense_rank().over(Window.orderBy(col('TotalCount').desc()))) \
    .withColumn('cum_freq', sum('frequency').over(Window.orderBy(col('overall_rank').asc())))

df_names_agg_stats.show(10, False)

+-----------+----------+---------+---------------------+------------+---------------+--------------------+
|FirstName  |TotalCount|YearCount|frequency            |overall_rank|freq_dense_rank|cum_freq            |
+-----------+----------+---------+---------------------+------------+---------------+--------------------+
|Michael    |3955521   |74       |0.014640494164521733 |1           |1              |0.014640494164521733|
|James      |2938863   |74       |0.010877557369011272 |2           |2              |0.025518051533533006|
|David      |2925435   |74       |0.010827856569637133 |3           |3              |0.036345908103170135|
|John       |2781375   |74       |0.010294650049095085 |4           |4              |0.04664055815226522 |
|Robert     |2601926   |74       |0.009630458900235235 |5           |5              |0.056271017052500455|
|William    |2170891   |74       |0.008035077305192604 |6           |6              |0.06430609435769306 |
|Christopher|2043593   |74       |0.0

In [59]:
df_a = df_names_agg_stats
df_b = df_names_agg_stats
df_comb = df_a.alias('a').join(df_b.alias('b'), on = col('a.overall_rank') == col('b.overall_rank') - 1, how='left') \
    .select(col('a.FirstName').alias('first_name'),
            col('a.TotalCount').alias('total_count'),
            col('a.frequency'),
            col('a.overall_rank'),
            col('a.cum_freq').alias('profile_lower_bound'),
            coalesce(col('b.cum_freq'), lit(1.0)).alias('profile_upper_bound'),
    )

df_comb.show(10, False)
df_comb.orderBy('overall_rank', ascending=False).show(10, False)
df_comb.count()


+-----------+-----------+---------------------+------------+--------------------+--------------------+
|first_name |total_count|frequency            |overall_rank|profile_lower_bound |profile_upper_bound |
+-----------+-----------+---------------------+------------+--------------------+--------------------+
|Michael    |3955521    |0.014640494164521733 |1           |0.014640494164521733|0.025518051533533006|
|James      |2938863    |0.010877557369011272 |2           |0.025518051533533006|0.036345908103170135|
|David      |2925435    |0.010827856569637133 |3           |0.036345908103170135|0.04664055815226522 |
|John       |2781375    |0.010294650049095085 |4           |0.04664055815226522 |0.056271017052500455|
|Robert     |2601926    |0.009630458900235235 |5           |0.056271017052500455|0.06430609435769306 |
|William    |2170891    |0.008035077305192604 |6           |0.06430609435769306 |0.07187000601210154 |
|Christopher|2043593    |0.007563911654408475 |7           |0.07187000601

97588

In [63]:
# Export and trim the column values down for I/O efficency during generation of data.
df_fnl = df_comb.select('first_name', 'overall_rank', 'profile_lower_bound', 'profile_upper_bound')
df_fnl.write.parquet(os.path.join(s_spark_file_server_root, 'SSA_FirstNames_Stats'), mode='overwrite')